## Lab5 Goal 2.2.1 ResNet Solution Augmented

In [ ]:
#only if you get restart karnell with plt.show
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torchvision import models

from random import randint

In [ ]:
# Step 1: Define image transformations
transform = transforms.Compose([
    transforms.Grayscale(),              # Ensure single channel
    #transforms.Resize((28, 28)),  # Resize if necessary
    transforms.RandomRotation(15),             # Rotate ±15 degrees
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Small shif  
    transforms.ToTensor(),        # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize grayscale images
])


In [ ]:
# Step 2: Load dataset from folder
dataset_path = 'ImgDataSet/train'  # Replace with your actual path
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Step 3: Split into train and test sets (e.g., 80/20)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [ ]:
# Step 4: Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Step 5: Check class labels
print("Classes:", full_dataset.classes)


In [ ]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model (ResNet18 pretrained model)
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # Load pretrained ResNet18
        self.base_model = models.resnet18(pretrained=True)

        # Adapt input layer for grayscale images (1 channel instead of 3)
        self.base_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Replace the final fully connected layer
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

model = NeuralNetwork().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    loss_ret=[]
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            loss_ret.append(loss)

    return loss_ret

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return [test_loss, correct]

In [ ]:
epochs = 10
loss=[]
acc_test=[]
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss.append(train(train_dataloader, model, loss_fn, optimizer))
    acc_test.append(test(test_dataloader, model, loss_fn))
print("Done!")

In [ ]:
# plot achieved accuracy after each epoch
x = list(range(1,epochs+1))
plt.plot(x, acc_test)
plt.xlabel("Epoch")
plt.plot(x, acc_test, label=['Loss', 'Acuracy/100'])
plt.legend(loc='upper right', title='Data Series')
plt.show()

In [ ]:
torch.save(model.state_dict(), "model_ResNet.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model_ResNet.pth", weights_only=True))

In [ ]:
model.eval()

for i in range(30):
    n= randint(0, test_size-1)
    x, y = test_dataset[n][0], test_dataset[n][1]
    with torch.no_grad():
        #x = x.to(device)
        x = x.unsqueeze(0).to(device)  # Shape becomes [1, 1, 28, 28]
        pred = model(x)
        print(f'"Test index: "{n}", Predicted: "{pred[0].argmax(0)}", Actual: "{y}"')